<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/analytics_vidhya/ml_summer_training_2022/notebooks/03_catboost_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade catboost
!pip install --upgrade optuna

In [2]:
import os
import gc
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import scipy.stats as st
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

import catboost
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [3]:
#remove cell to run future versions
assert optuna.__version__ == '2.10.1', f'Change in Optuna version. Original notebook version: 2.10.1'
assert catboost.__version__ == '1.0.6', f'Change in CatBoost version. Original notebook version: 1.0.6'

In [4]:
SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [5]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    GPU = True
except Exception:
    GPU = False

print(f'GPU available: {GPU}')

GPU available: False


**Data setup**

In [6]:
data_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/'

train_all_url = data_url + 'processed/all_features/train.csv'
test_all_url = data_url + 'processed/all_features/test.csv'
train_all = pd.read_csv(train_all_url)
test_all = pd.read_csv(test_all_url)

train_original_url = data_url + 'processed/original_filled/train.csv'
test_original_url = data_url + 'processed/original_filled/test.csv'
train_original = pd.read_csv(train_original_url)
test_original = pd.read_csv(test_original_url)

train_selected_url = data_url + 'processed/selected_features/train.csv'
test_selected_url = data_url + 'processed/selected_features/test.csv'
train_selected = pd.read_csv(train_selected_url)
test_selected = pd.read_csv(test_selected_url)

In [7]:
target_url = data_url + 'raw/train.csv'
target = pd.read_csv(target_url, usecols=['loan_default']).iloc[:, 0] #converted to series

test_index_url = data_url + 'raw/test.csv'
test_index = pd.read_csv(test_index_url, usecols=['loan_id']).iloc[:, 0]

**Categorical data conversion**

In [8]:
categorical_features = ['proof_submitted', 'last_delinq_none', 
                        'education_fill_1', 'education_fill_2', 
                        'age_range', 'loan_amount_range', 
                        'asset_cost_range', 'no_of_loans_range', 
                        'no_of_curr_loans_range', 'proof_is_aadhar']

train_all[categorical_features] = train_all[categorical_features].astype('category')
test_all[categorical_features] = test_all[categorical_features].astype('category')

In [9]:
cat_cols = [f for f in train_original.columns if f in categorical_features]
train_original[cat_cols] = train_original[cat_cols].astype('category')
test_original[cat_cols] = test_original[cat_cols].astype('category')

In [10]:
cat_cols = [f for f in train_selected.columns if f in categorical_features]
train_selected[cat_cols] = train_selected[cat_cols].astype('category')
test_selected[cat_cols] = test_selected[cat_cols].astype('category')

# Baselines

In [11]:
def fit_baseline(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, shuffle=True, stratify=target, random_state=SEED)
    
    cat_features = list(X.select_dtypes(include='category').columns)
    baseline = CatBoostClassifier()
    baseline.fit(
        X_train, y_train, 
        eval_set=[(X_val, y_val)],
        cat_features=cat_features,
        verbose=False)
    
    preds = baseline.predict(X_val)
    print(classification_report(y_val, preds))

**All features**

In [12]:
fit_baseline(train_all, target)

              precision    recall  f1-score   support

           0       0.61      0.88      0.72       840
           1       0.45      0.14      0.22       560

    accuracy                           0.59      1400
   macro avg       0.53      0.51      0.47      1400
weighted avg       0.54      0.59      0.52      1400



**Original features**

In [13]:
fit_baseline(train_original, target)

              precision    recall  f1-score   support

           0       0.60      0.95      0.74       840
           1       0.44      0.06      0.11       560

    accuracy                           0.59      1400
   macro avg       0.52      0.51      0.42      1400
weighted avg       0.54      0.59      0.49      1400



**Selected features**

In [14]:
fit_baseline(train_selected, target)

              precision    recall  f1-score   support

           0       0.60      0.92      0.73       840
           1       0.44      0.09      0.16       560

    accuracy                           0.59      1400
   macro avg       0.52      0.51      0.44      1400
weighted avg       0.54      0.59      0.50      1400



# Hyperparameter tuning

In [15]:
def objective(trial, data, base_params):
    
    scores = []
    X, y = data
    cat_features = list(X.select_dtypes(include='category').columns)
    
    #Defining hyperparameter search space
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3, step=0.05),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 10, step=0.1),
        'depth': trial.suggest_int('depth', 3, 16),
        'rsm': trial.suggest_float('rsm', 0.5, 1.0, step=0.05), #colsample_bylevel
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 4, 10),
        'random_strength': trial.suggest_float('random_strength', 0.2, 5),
        'bootstrap_type': trial.suggest_categorical(
            'bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'scale_pos_weight': trial.suggest_float(
            'scale_pos_weight', 1, 1.5, step=0.025)
    }
    #conditional hyperparameters
    if param_grid['bootstrap_type'] == 'Bayesian':
        param_grid['bagging_temperature'] = trial.suggest_int('bagging_temperature', 0, 10)
    else:
        param_grid['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
    
    #cross-validating for each trial
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = CatBoostClassifier(**base_params, **param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            cat_features=cat_features,
            early_stopping_rounds=50,
            verbose=False)
        preds = model.predict(X_val)
        scores.append(f1_score(y_val, preds, average='macro'))
    
    return np.mean(scores)

In [16]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction)
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True)
    
    return study

# Cross-validation

In [17]:
def evaluate_model(data, model_params, verbose=True):
    
    preds_test = []
    scores_f1 = [] #validation set weighted-f1 scores
    scores_acc = [] #validation set accuracy
    
    X, X_test, y = data
    cat_features = list(X.select_dtypes(include='category').columns)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = CatBoostClassifier(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            cat_features=cat_features,
            early_stopping_rounds=50,
            verbose=False)
        
        preds_val = model.predict(X_val)
        preds_test.append(model.predict(X_test))
        
        scores_acc.append(accuracy_score(y_val, preds_val))
        scores_f1.append(f1_score(y_val, preds_val, average='macro'))
    
    if verbose:
        scores_df = pd.DataFrame.from_dict({
            'Accuracy': scores_acc,
            'Macro-f1': scores_f1
        })
        scores_df.index.name = 'Fold'
        display(scores_df)
    
    print(f'Average Accuracy = {np.mean(scores_acc):.4f} (with std = {np.std(scores_acc):.4f})')
    print(f'Average Macro-f1 = {np.mean(scores_f1):.4f} (with std = {np.std(scores_f1):.4f})\n')
    
    preds_test = st.mode(np.column_stack(preds_test), axis=1).mode
    return preds_test.ravel()

In [18]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'n_estimators': 2000,
        # 'boosting_type': 'Ordered',
        'loss_function': 'Logloss',
        'eval_metric': 'F1',
        'early_stopping_rounds': 50,
        # 'task_type': 'GPU',
        'use_best_model': True,
        'one_hot_max_size': 5,
        'verbose': False,
        'random_seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='maximize')
    print(f'Best trial: {study.best_trial.number} -> Best value (f1-macro): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    preds_test = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params)
    
    return preds_test

In [19]:
%%time
preds_all = run_experiment((train_all, test_all, target), 100)

---------------Hyperparameter tuning---------------
Best trial: 79 -> Best value (f1-macro): 0.57523
Best hyperparameters:
learning_rate        - 0.3
l2_leaf_reg          - 2.2
depth                - 6
rsm                  - 0.9
max_ctr_complexity   - 10
random_strength      - 4.765553629866167
bootstrap_type       - MVS
scale_pos_weight     - 1.375
subsample            - 0.7605240779415192
-----------------Cross-validation------------------


,Accuracy,Macro-f1
Fold,,
0,0.5721,0.5652
1,0.5857,0.5746
2,0.5800,0.5664
3,0.6014,0.5917
4,0.5864,0.5782


Average Accuracy = 0.5851 (with std = 0.0096)
Average Macro-f1 = 0.5752 (with std = 0.0096)

CPU times: user 11min 43s, sys: 3min 25s, total: 15min 9s
Wall time: 9min 21s


In [20]:
%%time
preds_original = run_experiment((train_original, test_original, target), 100)

---------------Hyperparameter tuning---------------
Best trial: 56 -> Best value (f1-macro): 0.58022
Best hyperparameters:
learning_rate        - 0.2
l2_leaf_reg          - 10.0
depth                - 3
rsm                  - 0.55
max_ctr_complexity   - 9
random_strength      - 3.7758301899738256
bootstrap_type       - Bayesian
scale_pos_weight     - 1.35
bagging_temperature  - 6
-----------------Cross-validation------------------


,Accuracy,Macro-f1
Fold,,
0,0.6050,0.5921
1,0.5793,0.5675
2,0.5950,0.5806
3,0.5886,0.5770
4,0.5957,0.5839


Average Accuracy = 0.5927 (with std = 0.0085)
Average Macro-f1 = 0.5802 (with std = 0.0081)

CPU times: user 12min 38s, sys: 4min 59s, total: 17min 38s
Wall time: 10min 32s


In [21]:
%%time
preds_selected = run_experiment((train_selected, test_selected, target), 100)

---------------Hyperparameter tuning---------------
Best trial: 92 -> Best value (f1-macro): 0.57814
Best hyperparameters:
learning_rate        - 0.2
l2_leaf_reg          - 4.300000000000001
depth                - 4
rsm                  - 0.8
max_ctr_complexity   - 5
random_strength      - 1.944248201153776
bootstrap_type       - Bernoulli
scale_pos_weight     - 1.35
subsample            - 0.8071180728396585
-----------------Cross-validation------------------


,Accuracy,Macro-f1
Fold,,
0,0.6036,0.5923
1,0.5757,0.5671
2,0.5964,0.5785
3,0.5936,0.5820
4,0.5800,0.5709


Average Accuracy = 0.5899 (with std = 0.0104)
Average Macro-f1 = 0.5781 (with std = 0.0088)

CPU times: user 11min 28s, sys: 3min 31s, total: 15min
Wall time: 9min 19s


# Generating submission files

In [22]:
sub_all = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_all
})
sub_all.to_csv('03_sub_all.csv', index=False)

In [23]:
sub_original = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_original
})
sub_original.to_csv('03_sub_original.csv', index=False)

In [24]:
sub_selected = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_selected
})
sub_selected.to_csv('03_sub_selected.csv', index=False)